- This notebook needs to be saved in the state "Restart Kernel and clear output of all Cells"
- The "Restart Kernel and run all Cells" should only be run once
- This is because the html "\<div\>" nodes can somehow easily get mixed up and then the blockly cells are not rendered 

In [ ]:
# ! pip install python_hiccup

In [ ]:
# ! pip install edn_format

In [ ]:
# ! pip install hissp

In [ ]:
# ! pip install sympy

In [ ]:
%%html
<script src="https://unpkg.com/blockly/blockly_compressed.js"></script>

In [ ]:
import time
time.sleep(10)

In [ ]:
from functools import partial
import edn_format

def walk(inner, outer, coll):
    if isinstance(coll, edn_format.immutable_list.ImmutableList):
        return outer([inner(e) for e in coll])
    elif isinstance(coll, edn_format.immutable_dict.ImmutableDict):
        return outer(dict([inner(e) for e in iter(coll.items())]))
    elif isinstance(coll, tuple):
        return outer(tuple([inner(e) for e in coll]))
    else:
        return outer(coll)

def postwalk(fn, coll):
    return walk(partial(postwalk, fn), fn, coll)

In [ ]:
def simpleExecute(coll):
    if isinstance(coll, edn_format.edn_lex.Symbol):
        return globals().get(str(coll))
    elif isinstance(coll, edn_format.edn_lex.Keyword):
        return str(coll)[1:]
    elif isinstance(coll, tuple):
        f = coll[0]
        if isinstance(f, str):
            return coll
        else:
            return f(*coll[1:])
    else:
        return coll

def tT(*d):
    return d
    
def ednToHissp(edn):
    if isinstance(edn, edn_format.edn_lex.Symbol):
        if edn == edn_format.edn_lex.Symbol("if-else"):
            return "ifQzH_else"
        else:
            return str(edn)    
    elif isinstance(edn, edn_format.edn_lex.Keyword):
        return ("quote", str(edn)[1:])
    elif isinstance(edn, str):
        return ("quote",edn)
    elif isinstance(edn, list):
        return tuple(["tT"] + edn)
    elif isinstance(edn, dict):
        return ("dict", tuple(["tT"] + [("tT",) + e for e in iter(edn.items())]))
    else:
        return edn

replace_def = lambda s:('defun', s[1][0], s[1][1:], s[2]) if s[0]=="def" else s
ednTxtToData = lambda s: replace_def(postwalk(ednToHissp, edn_format.loads(s)))

In [ ]:
from IPython.core.magic import (Magics, magics_class, cell_magic)
from IPython.display import display, HTML
from python_hiccup.html import render
from hissp import readerless
from hissp import _macro_
from uuid import uuid4

def html(hiccupString):
    display(HTML(render(hiccupString)))
    return None
    
def displayHTML(s):
    divid = str(uuid4())
    erg = display(HTML("<div id='" + divid + "'>error</div><script>document.getElementById('" + divid + "').innerHTML = " + s + " </script>"))
    
@magics_class
class MyMagics(Magics):
    @cell_magic
    def inter(self, line, cell):
        return postwalk(simpleExecute, edn_format.loads(cell))
        
    @cell_magic
    def hissp(self, line, cell):
        erg=eval(readerless(ednTxtToData(cell)))
        return erg

    @cell_magic
    def hissp0(self, line, cell):
        erg=ednTxtToData(cell)
        return erg
        
    @cell_magic
    def hissp1(self, line, cell):
        erg=print(readerless(ednTxtToData(cell)))
        return erg

    @cell_magic
    def scheme(self, line, cell):
        erg = displayHTML("eval(expressionToJs('" + cell.replace("\n", "") +"'))")
        return None
        
    @cell_magic
    def scheme0(self, line, cell):
        erg = displayHTML("expressionToJs('" + cell.replace("\n", "") +"')")
        return None

In [ ]:
from IPython import get_ipython
get_ipython().register_magics(MyMagics)

In [ ]:
%%hissp
(define f (lambda (name) {:key name}))

In [ ]:
f("ha")

In [ ]:
%%inter
(html [:h1 "Hello world!"])

In [ ]:
%%inter
(html [:svg {:width 200 :height 100} [:circle {:cx 50 :cy 50 :r 25 :style "fill:red"}]])

In [ ]:
%%hissp
(html [:svg {:width 200 :height 100} [:circle {:cx 50 :cy 50 :r 25 :style "fill:blue"}]])

In [ ]:
def subs(s, m):
    return s[m:]

def plus(a, b):
    return a + b

num = 1

In [ ]:
%%inter
(html [:h3 (subs (subs "hi_hiccup" (plus 1 num)) 1)])

In [ ]:
%%inter
[:h3 (subs (subs "hi_hiccup" (plus 1 num)) 1)]

In [ ]:
%%hissp
(html [:h3 (subs (subs "hi_hissp" (plus 1 num)) 1)])

In [ ]:
%%hissp
[:h3 (subs (subs "hi_hissp" (plus 1 num)) 1)]

In [ ]:
%%hissp0
(html [:h3 (subs (subs "hi_hissp" (plus 1 num)) 1)])

In [ ]:
%%hissp1
(html [:h3 (subs (subs "hi_hissp" (plus 1 num)) 1)])

In [ ]:
%%html
<script>
Blockly.defineBlocksWithJsonArray([
{
             "type": "list-h-2",
             "message0": " %1 \u007C %2",
             "args0": [
                 {
                     "type": "input_value",
                     "name": "args-1",
                 },
                 {
                     "type": "input_value",
                     "name": "args-2"
                 }
             ],
             "inputsInline": true,
             "output": null,
             "colour": 70,
             "tooltip": "",
             "helpUrl": ""
         },
         {
             "type": "num",
             "message0": "%1",
             "args0": [
                 {
                     "type": "field_input",
                     "name": "nummer",
                     "text": "0.0"
                 }
             ],
             "output": null,
             "colour": "#A65C81",
             "tooltip": "",
             "helpUrl": ""
         },
         {
             "type": "funs-h-2-inp",
             "message0": "%1 %2",
             "args0": [
                 {
                     "type": "field_input",
                     "name": "kopf",
                     "text": ""
                 },
                 {
                     "type": "input_value",
                     "name": "args-2"
                 }
             ],
             "inputsInline": true,
             "output": null,
             "colour": 270,
             "tooltip": "",
             "helpUrl": ""
         },
         {
             "type": "funs-h-3-inp",
             "message0": "%1 %2 %3",
             "args0": [
                 {
                     "type": "field_input",
                     "name": "kopf",
                     "text": ""
                 },
                 {
                     "type": "input_value",
                     "name": "args-2"
                 },
                 {
                     "type": "input_value",
                     "name": "args-3"
                 }
             ],
             "inputsInline": true,
             "output": null,
             "colour": 140,
             "tooltip": "",
             "helpUrl": ""
         },
    {
             "type": "infi-h-3-inp",
             "message0": "%1 %2 %3",
             "args0": [
                 {"type": "input_value",
                  "name": "args-2"},
                 {"type": "field_input",
                  "name": "kopf",
                  "text": ""
                 },
                 {"type": "input_value",
                  "name": "args-3"}],
             "inputsInline": true,
             "output": null,
             "colour": 140,
             "tooltip": "",
             "helpUrl": ""}
     ]);
</script>

In [ ]:
import time
time.sleep(5)

In [ ]:
%%html
<script>
     var toolbox = {
         "kind": "categoryToolbox",
         "contents": [
             {
                 "kind": "category",
                 "name": ">",
                 "contents": [
                     {"kind": "block", "type": "num"},
                     {"kind": "block", "type": "funs-h-2-inp"},
                     {"kind": "block", "type": "funs-h-3-inp"},
                 ]
     }]}
</script>

In [ ]:
%%html
<div id="blocklyDiv1" style="height: 90%"></div>
<div>Footer</div><h1>2</h1><h1>3</h1><h1>4</h1><h1>5</h1>
<script>
var workspace1 = Blockly.inject('blocklyDiv1', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
%%html
<script>
var xs = "<xml><block id=\"funs-h-2-inpid1\" type=\"funs-h-2-inp\"><field name=\"kopf\">b</field><value name=\"args-2\"><block id=\"num2-funs-h-2-inpid1\" type=\"num\"><field name=\"nummer\">1</field></block></value></block></xml>"
var xmlDom = Blockly.utils.xml.textToDom(xs);
Blockly.Xml.clearWorkspaceAndLoadFromXml(xmlDom, workspace1)
</script>

In [ ]:
from sympy import symbols,diff,log
x=symbols("x")
base=symbols("base")

def div(x,y):
    return x/y

In [ ]:
%%hissp
(def (h_logarithm base x)
 (div (log x) (log base)))

In [ ]:
%%hissp
(h_logarithm 8.0 64.0)

In [ ]:
%%hissp
(def (h_log_base base) 
 (partial h_logarithm base))

In [ ]:
%%hissp
((h_log_base 8.0) 64.0)

In [ ]:
%%hissp
(diff ((h_log_base 8) x) x)

In [ ]:
%%html
<script src="https://kloimhardt.github.io/blog/js/emmy.js/build/emmy_bundle.js"></script>
<script>
     var loadEnv = (name) => {
         window[name] = emmy[name];
         return name;
     }
    </script>

In [ ]:
%%html
<script>
var symbol = emmy.symbol;

var specialchars =
[["", /\:/g],
 ["__gt_", /->/g],
 ["symbol_$1", /\'(\w+)/g],
 ["$1_dot_$2", /(\d+)\.(\d+)/g],
 [" minus_$2", /(\s+)\-(\w+)/g],
 ["$1_", /(\w+)\-/g],
 ["[ / $1 $2 ],", /(\w+)\/(\d+)/g]];

var post_specialchars =
[["$1.$2", /(\d+)_dot_(\d+)/g],
 ["$1e-$2", /(\d+)e_(\d+)/],
 ["-$1", /minus_(\w+)/g],
 //case symbol with hat 'v_r^x
 ["symbol_$1^$2", /symbol_(\w+)expt(\w+)/]]

var mathfns =
[["div", /\//g],
 ["mul", /\*/g],
 ["sub", /\-/g],
 ["add", /\+/g],
 ["expt", /\^/g]]

mathfns.map(x=> x[0]).map(loadEnv);

var replaceMath = (txt) =>
specialchars.concat(mathfns)
.reduce((s,r) => s.replace(r[1],r[0]), txt);

var swapFirst = (j) =>
j.constructor == Array && mathfns.map(x=> x[0]).includes(j[1])
? [j[1], j[0]].concat(j.slice(2)).map(swapFirst)
: j.constructor == Array
? j.map(swapFirst)
: j;

var insertCommas = (txt) =>
txt.replace(/(\w+)/g,'"$1",')
.replace(/\,\s+\]/g," ]");

var makeBrackets = (txt) =>
txt.trim()
.replace(/\(/g,"[ ")
           .replace(/\)/g," ],")
.replace(/,$/,"");

var textToJson = (txt) =>
insertCommas(replaceMath(makeBrackets(txt)));

var drv = emmy.D;

var replaceD = (j) =>
j.constructor == Array
? j.map(replaceD)
: j.constructor == String && j == "D"
? "drv"
: j.constructor == String
? post_specialchars.reduce((s,r) => s.replace(r[1],r[0]), j)
: j;

["to_infix", "simplify"].map(loadEnv);

var preAmble = (j) =>
j[0] != "define" && j[0] != "def"
? ["to_infix", j]
: j;

var identity = x => x;
var retrn = x => x;

var edgeCases = (j) =>
(j[0] === "define" || j[0] === "def")
&& j[2].constructor == Array && j[2][0] === "let"
&& j[1].constructor == String
? ["identity", "'let not allowed in variable definition'"]
: j;

var letFlat2 = (j) =>
j[0] === "let"
? j[1].concat(letFlat2(j[2]))
: [j];

var constructLet = (j) =>
["let", j.slice(0, j.length - 1), j[j.length - 1]]

var letFlat = (j) =>
j.constructor == Array && j[0] === "let"
? constructLet(letFlat2(j))
: j.constructor == Array
? j.map(letFlat)
: j;

var modifyJson = (j) =>
preAmble(replaceD(letFlat(edgeCases(swapFirst(j)))));

var makeFun = (j, callBack) =>
"var " + j[1][0] + " = (" + j[1].slice(1) + ") => "
+ callBack(j[2]) +";";

var makeFunFun = (j, callBack) =>
"var " + j[1][0][0] + " = (" + j[1][0].slice(1) + ") => "
+ "(" + j[1].slice(1) +") => " + callBack(j[2]) + ";";

var smap = (f, v) =>
v.length === 1
? f(v[0]) +";"
: f(v[0]) +"; "  + smap(f, v.slice(1));

var makeLet = (j, callBack) =>
"{" + smap((l) => "let " + l[0] + " = " + callBack(l[1]), j[1])
+ " return " +  callBack(j[2]) + "; }"

var jsonToJs = (j) =>
j.constructor == Array && j[0] === "let"
? makeLet(j, jsonToJs)
:j.constructor == Array && (j[0] === "define" || j[0] === "def")
&& j[1].constructor == Array
&& j[1][0].constructor == Array
? makeFunFun(j, jsonToJs)
:j.constructor == Array && (j[0] === "define" || j[0] === "def")
&& j[1].constructor == Array
? makeFun(j, jsonToJs)
: j.constructor == Array && j[0] === "lambda"
? "(" + j[1] + ") => " + jsonToJs(j[2])
:j.constructor == Array && (j[0] === "define" || j[0] === "def")
&& j[1].constructor == String
? "var " + j[1] + " = " + jsonToJs(j[2]) + ";"
:j.constructor == String && j.substring(0, 7) == "symbol_"
? 'symbol("' + j.substring(7, j.length) + '")'
:j.constructor == Array
? jsonToJs(j[0]) + "("  +  j.slice(1).map(jsonToJs) + ")"
: j;

var expressionToJs = (expr) =>
jsonToJs(modifyJson(JSON.parse(textToJson(expr))));

["sin", "cos", "pi", "velocity", "dot_product",
        "up", "nth", "Gamma", "compose", "literal_function",
        "definite_integral", "coordinate", "minimize",
        "linear_interpolants", "multidimensional_minimize",
        "make_path", "count",
        "square", "partial", "ref", "sqrt", "atan", "down",
        "solve_linear_left", "state_advancer", "cross_product",
        "Rx", "Ry", "Rz", "osculating_path", "log"].map(loadEnv);

var show_expression = simplify;
var vector_length = count;
var velocities = velocity;
var coordinates = coordinate;
var time = state => nth(state, 0);
var get = (obj, field) => obj[field.toString()]; null;
</script>

In [ ]:
%%scheme
(+ 4 5 6)

In [ ]:
%%html
<script>var x = symbol('x'); base=symbol('base') </script>

In [ ]:
%%html
<div id="blocklyDiv3" style="height: 90%"></div>
<div></div><h1>2</h1>
<script>
var workspace3 = Blockly.inject('blocklyDiv3', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
diff(log(x),x)

In [ ]:
%%html
<script>
var xs3 = "<xml><block id=\"funs-h-3-inp-10-10\" type=\"funs-h-3-inp\" x=\"10\" y=\"10\"><field name=\"kopf\">diff</field><value name=\"args-2\"><block id=\"funs-h-2-inp2-funs-h-3-inp-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">log</field><value name=\"args-2\"><block id=\"num2-funs-h-2-inp2-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">x</field></block></value></block></value><value name=\"args-3\"><block id=\"num3-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">x</field></block></value></block></xml>"

Blockly.Xml.clearWorkspaceAndLoadFromXml(Blockly.utils.xml.textToDom(xs3), workspace3)
</script>

In [ ]:
%%html
<div id="blocklyDiv2" style="height: 90%"></div>
<div></div><h1>2</h1><h1>3</h1>
<script>
var workspace2 = Blockly.inject('blocklyDiv2', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
def logarithm(base,x):
    return log(x)/log(base)

In [ ]:
%%scheme
(def (logarithm base x) 
 (retrn ((log x) / (log base))))

In [ ]:
%%html
<script>
var xs2 = "<xml><block id=\"funs-h-3-inp-10-10\" inline=\"false\" type=\"funs-h-3-inp\" x=\"10\" y=\"10\"><field name=\"kopf\">def</field><value name=\"args-2\"><block id=\"funs-h-3-inp2-funs-h-3-inp-10-10\" type=\"funs-h-3-inp\"><field name=\"kopf\">logarithm</field><value name=\"args-2\"><block id=\"num2-funs-h-3-inp2-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">base</field></block></value><value name=\"args-3\"><block id=\"num3-funs-h-3-inp2-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">x</field></block></value></block></value><value name=\"args-3\"><block id=\"infi-h-3-inp3-funs-h-3-inp-10-10\" type=\"infi-h-3-inp\"><field name=\"kopf\">/</field><value name=\"args-2\"><block id=\"funs-h-2-inp2-infi-h-3-inp3-funs-h-3-inp-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">log</field><value name=\"args-2\"><block id=\"num2-funs-h-2-inp2-infi-h-3-inp3-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">x</field></block></value></block></value><value name=\"args-3\"><block id=\"funs-h-2-inp3-infi-h-3-inp3-funs-h-3-inp-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">log</field><value name=\"args-2\"><block id=\"num2-funs-h-2-inp3-infi-h-3-inp3-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">base</field></block></value></block></value></block></value></block></xml>"
Blockly.Xml.clearWorkspaceAndLoadFromXml(Blockly.utils.xml.textToDom(xs2), workspace2)
</script>

In [ ]:
%%html
<div id="blocklyDiv5" style="height: 90%"></div>
<div></div><h1>2</h1>
<script>
var workspace5 = Blockly.inject('blocklyDiv5', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
logarithm(8.0, 64.0)

In [ ]:
%%scheme
(logarithm 8.0 64.0)

In [ ]:
%%html
<script>
var xs5 = "<xml><block id=\"funs-h-3-inp-10-10\" type=\"funs-h-3-inp\" x=\"10\" y=\"10\"><field name=\"kopf\">logarithm</field><value name=\"args-2\"><block id=\"num2-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">8.0</field></block></value><value name=\"args-3\"><block id=\"num3-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">64.0</field></block></value></block></xml>"
Blockly.Xml.clearWorkspaceAndLoadFromXml(Blockly.utils.xml.textToDom(xs5), workspace5)
</script>

In [ ]:
%%html
<div id="blocklyDiv6" style="height: 90%"></div>
<div></div><h1>2</h1><h1>3</h1>
<script>
var workspace6 = Blockly.inject('blocklyDiv6', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
def log_base(base):
    return partial(logarithm, base)

In [ ]:
%%scheme
(def (log_base base) 
 (retrn (partial logarithm base)))

In [ ]:
%%html
<script>
var xs6 = "<xml><block id=\"funs-h-3-inp-10-10\" inline=\"false\" type=\"funs-h-3-inp\" x=\"10\" y=\"10\"><field name=\"kopf\">def</field><value name=\"args-2\"><block id=\"funs-h-2-inp2-funs-h-3-inp-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">log_base</field><value name=\"args-2\"><block id=\"num2-funs-h-2-inp2-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">base</field></block></value></block></value><value name=\"args-3\"><block id=\"funs-h-2-inp3-funs-h-3-inp-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">return</field><value name=\"args-2\"><block id=\"funs-h-3-inp2-funs-h-2-inp3-funs-h-3-inp-10-10\" type=\"funs-h-3-inp\"><field name=\"kopf\">partial</field><value name=\"args-2\"><block id=\"num2-funs-h-3-inp2-funs-h-2-inp3-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">logarithm</field></block></value><value name=\"args-3\"><block id=\"num3-funs-h-3-inp2-funs-h-2-inp3-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">base</field></block></value></block></value></block></value></block></xml>"
Blockly.Xml.clearWorkspaceAndLoadFromXml(Blockly.utils.xml.textToDom(xs6), workspace6)
</script>

In [ ]:
%%html
<div id="blocklyDiv7" style="height: 90%"></div>
<div></div><h1>2</h1><h1>3</h1>
<script>
var workspace7 = Blockly.inject('blocklyDiv7', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
log_base(8.0)(64.0)

In [ ]:
%%scheme
((log_base 8.0) 64.0)

In [ ]:
%%html
<script>
var xs7 = "<xml><block id=\"list-h-2-10-10\" type=\"list-h-2\" x=\"10\" y=\"10\"><value name=\"args-1\"><block id=\"funs-h-2-inp1-list-h-2-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">log_base</field><value name=\"args-2\"><block id=\"num2-funs-h-2-inp1-list-h-2-10-10\" type=\"num\"><field name=\"nummer\">8.0</field></block></value></block></value><value name=\"args-2\"><block id=\"num2-list-h-2-10-10\" type=\"num\"><field name=\"nummer\">64.0</field></block></value></block></xml>"
Blockly.Xml.clearWorkspaceAndLoadFromXml(Blockly.utils.xml.textToDom(xs7), workspace7)
</script>

In [ ]:
%%html
<div id="blocklyDiv8" style="height: 90%"></div>
<div></div><h1>2</h1><h1>3</h1>
<script>
var workspace8 = Blockly.inject('blocklyDiv8', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
diff(log_base(8.0)(x), x)

In [ ]:
%%html
<script>
//(diff ((log_base 8.0) x) x)
var xs8 = "<xml><block id=\"funs-h-3-inp-10-10\" type=\"funs-h-3-inp\" x=\"10\" y=\"10\"><field name=\"kopf\">diff</field><value name=\"args-2\"><block id=\"list-h-22-funs-h-3-inp-10-10\" type=\"list-h-2\"><value name=\"args-1\"><block id=\"funs-h-2-inp1-list-h-22-funs-h-3-inp-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">log_base</field><value name=\"args-2\"><block id=\"num2-funs-h-2-inp1-list-h-22-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">8.0</field></block></value></block></value><value name=\"args-2\"><block id=\"num2-list-h-22-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">x</field></block></value></block></value><value name=\"args-3\"><block id=\"num3-funs-h-3-inp-10-10\" type=\"num\"><field name=\"nummer\">x</field></block></value></block></xml>"
Blockly.Xml.clearWorkspaceAndLoadFromXml(Blockly.utils.xml.textToDom(xs8), workspace8)
</script>

In [ ]:
%%html
<div id="blocklyDiv9" style="height: 90%"></div>
<div></div><h1>2</h1><h1>3</h1>
<script>
var workspace9 = Blockly.inject('blocklyDiv9', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
%%scheme
((D (log_base 8)) x)

In [ ]:
%%html
<script>
var xs9 = "<xml><block id=\"list-h-2-10-10\" type=\"list-h-2\" x=\"10\" y=\"10\"><value name=\"args-1\"><block id=\"funs-h-2-inp1-list-h-2-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">D</field><value name=\"args-2\"><block id=\"funs-h-2-inp2-funs-h-2-inp1-list-h-2-10-10\" type=\"funs-h-2-inp\"><field name=\"kopf\">log_base</field><value name=\"args-2\"><block id=\"num2-funs-h-2-inp2-funs-h-2-inp1-list-h-2-10-10\" type=\"num\"><field name=\"nummer\">8</field></block></value></block></value></block></value><value name=\"args-2\"><block id=\"num2-list-h-2-10-10\" type=\"num\"><field name=\"nummer\">x</field></block></value></block></xml>"
Blockly.Xml.clearWorkspaceAndLoadFromXml(Blockly.utils.xml.textToDom(xs9), workspace9)
</script>

In [ ]:
%%scheme
(D log_base)

In [ ]:
diff(logarithm(base,x),x)

In [ ]:
try:
    diff(log_base(base),base)
except:
    print("cannot do")

In [ ]:

diff(log_base(8)(x),x)

In [ ]:
%%hissp
(def (num nummer)
 {:type :num :nummer nummer})

In [ ]:
%%hissp
(def (funx name argsvec)
  {:type :fun :subtype "funs-h" :kopf name :argsvec argsvec})

In [ ]:
funx("a", "b")

In [ ]:
def isTuple(t):
    return isinstance(t, tuple)

def first(coll):
    return coll[0]

def rest(coll):
    return coll[1:]

def map(f, coll):
    return (tuple (f(e) for e in coll))

def join_str(*strs):
    return "".join(map(str, strs))

def get(m, k):
    if isinstance(m, dict):
        if k in m:
            return m[k]
        else:
            return None
    else:
        return m[k]

def eq(a, b):
    return a==b

def inc(n):
    return n+1

def count(t):
    return len(t)

In [ ]:
count(rest(map(str, ("a","b","c"))))

In [ ]:
get({"a":1, "b":2}, "b")
get(("a", 1, "b", 2), 0)

In [ ]:
%%hissp
(def (parse l)
    (if-else (isTuple l)
        (funx (first l) (map parse (rest l)))
        (num l)))

In [ ]:
%%hissp
(get (get (parse ["a" "1"]) :argsvec) 0) 

In [ ]:
%%hissp
(def (blockmap type givenid inline)
 {:type type :id (join_str type givenid) :inline (str inline)})

In [ ]:
%%hissp
(def (gen m givenid)
  (cond
    (eq (get m :type) :num)
    [:block (blockmap "num" givenid False) [:field {:name "nummer"} (get m :nummer)]]
    (eq (get m :type) :fun)
    (gen (get (get m :argsvec) 0) givenid)))

In [ ]:
%%hissp
(gen (parse ["a" "1"]) "id")

In [ ]:
%%hissp
(def (genfun m givenid)
  (let (xml_block_type (join_str (get m :subtype) "-" (inc (count (get m :argsvec))) "-inp"))
    (let (bm (blockmap xml_block_type givenid (get m :inline)))
      (let (id (get bm :id))
        True))))

In [ ]:
%%hissp
(genfun (parse ["a" "1"]) "id")

In [ ]:
def pyfunction(x, base):
    return log(x,base)

In [ ]:
pyfunction(64,8)

In [ ]:
def retrn (x):
    return x

In [ ]:
%%hissp
(def (myfunction x base)
     (retrn (log x base)))

In [ ]:
myfunction(64,8)

In [ ]:
%%html
<div id="blocklyDiv20" style="height: 90%"></div>
<div>Footer</div><h1>2</h1><h1>3</h1>
<script>
var workspace3 = Blockly.inject('blocklyDiv20', {"toolbox": toolbox, "sounds": false});
</script>

In [ ]:
%%html
<script>
var xs20 = "<xml><block id=\"funs-h-3-inpid1\" inline=\"false\" type=\"funs-h-3-inp\"><field name=\"kopf\">def</field><value name=\"args-2\"><block id=\"funs-h-3-inp2-funs-h-3-inpid1\" type=\"funs-h-3-inp\"><field name=\"kopf\">myfunction</field><value name=\"args-2\"><block id=\"num2-funs-h-3-inp2-funs-h-3-inpid1\" type=\"num\"><field name=\"nummer\">x</field></block></value><value name=\"args-3\"><block id=\"num3-funs-h-3-inp2-funs-h-3-inpid1\" type=\"num\"><field name=\"nummer\">base</field></block></value></block></value><value name=\"args-3\"><block id=\"funs-h-2-inp3-funs-h-3-inpid1\" type=\"funs-h-2-inp\"><field name=\"kopf\">return</field><value name=\"args-2\"><block id=\"funs-h-3-inp2-funs-h-2-inp3-funs-h-3-inpid1\" type=\"funs-h-3-inp\"><field name=\"kopf\">log</field><value name=\"args-2\"><block id=\"num2-funs-h-3-inp2-funs-h-2-inp3-funs-h-3-inpid1\" type=\"num\"><field name=\"nummer\">x</field></block></value><value name=\"args-3\"><block id=\"num3-funs-h-3-inp2-funs-h-2-inp3-funs-h-3-inpid1\" type=\"num\"><field name=\"nummer\">base</field></block></value></block></value></block></value></block></xml>"
var xmlDom20 = Blockly.utils.xml.textToDom(xs20);
Blockly.Xml.clearWorkspaceAndLoadFromXml(xmlDom20, workspace3)
</script>